In [1]:
import json
import os
from collections import Counter
from itertools import combinations
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from scipy.sparse import hstack
from collections import defaultdict
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import io
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import FastText

In [2]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Concatenate, Add, Input
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.backend import tf

Using TensorFlow backend.


In [3]:
data = json.load(open('train.json'))

In [4]:
relations = tuple(data.keys())
len_relations = len(relations)
len_relations

64

Берем предобученные эмбеддинги Fasttext с их официального сайта/гитахаба (все все равно ведет сюда: https://fasttext.cc/docs/en/crawl-vectors.html)

In [5]:
path = "cc.en.300.bin"
word_vectors = FastText.load_fasttext_format(path)

In [6]:
len(word_vectors.wv['flight'])

300

Идея простая: возьмем эмбеддинги для каждого слова и прибавим к ним след.вектор:

[0,1,x,x,x,x,x]+[1,0,x,x,x,x,x]

первая часть (до плюса) будет отвечать за h (0,1), второй за t (1,0)

дальше у не-сущностей будет последовтаельность 0,0,0,1,0 (bilOu)

у начальной позиции 1,0,0,0,0 (Bilou)

и т.д.

итого у нас для каждого входного слова будет вектор длиной 314 (300 fastext, 14 для кодирования сущностей)

In [7]:
def create_bilou_tag(indices, i):
    bilou = {"b":0, 'i':1, 'l':2, 'o':3, 'u':4}
    result = np.zeros(len(bilou))
    for l in indices:
        if i in l:
            if len(l)==1 and l[0]==i:
                result[bilou['u']]=1
            else:
                if l[0]==i:
                    result[bilou['b']]=1
                elif l[-1]==i:
                    result[bilou["l"]]=1
                else:
                    result[bilou['i']]=1
    if (result == np.zeros(len(bilou))).all():
        result[bilou['o']]=1
    return result

In [8]:
def create_ht_vectors(h,t, length):
    h_indices = h[2]
    t_indices = t[2]
    result_vectors = []
    
    for i in range(length):
        h_vector = np.concatenate(([0.0, 1.0],create_bilou_tag(h_indices, i)))
        t_vector = np.concatenate(([1.0, 0.0],create_bilou_tag(t_indices, i)))
        result_vectors.append(np.concatenate((h_vector, t_vector)))
    return np.array(result_vectors)

In [9]:
def create_input_sequence(instance):
    sentence_array = []
    for token in instance['tokens']:
        try:
            sentence_array.append(word_vectors.wv[token])
        except KeyError as e:
            sentence_array.append(np.zeros(300))
    ht_vectors = create_ht_vectors(instance['h'], instance['t'], len(sentence_array))
    return np.concatenate((sentence_array,ht_vectors),axis=1)

In [41]:
x,y, instances = [],[], []
for i in data:
    for instance in data[i]:
        x.append(create_input_sequence(instance))
        
        output = np.zeros(len_relations)
        output[relations.index(i)]=1
        y.append(output)
        
        instances.append(instance)
x = pad_sequences(np.array(x))
y = np.array(y)
print(x.shape, y.shape)

(44800, 36, 314) (44800, 64)


In [14]:
embedding_size = 128
hidden_size = 128
out_size = len_relations
nb_epoch = 10
batch_size = 32
maxlen=36

#### Модель
Я посмотрела на контекст в ноутбуке семинара и ответ на вопрос, какую строить модель, пришел сам собой:

Возьмем LSTM и переведем вектора в новое пространство. Получим вектор-состяние LSTM и представим, что это наш прекрасный вектор предложения, который знает и о сущностях тоже и о всем смысле предложения.

А дальше сделаем классификатор: пропустим сконкатенированный вектор состояния через fully-connected layer и будем ожидать на выходе один из 64 классов.

Звучить красиво, а на деле..

In [12]:
def train_model(x, y):
    prem_input = Input(shape=(maxlen, 314), dtype='float32')
    a,h,c = LSTM(output_dim=hidden_size, return_sequences=False, return_state=True, inner_activation='sigmoid')(prem_input)
    state = Concatenate()([h,c])
    final_dense = Dense(out_size, activation='softmax')(state)
    model = Model(input=[prem_input], output=final_dense)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x, y, batch_size=batch_size, epochs=nb_epoch, validation_split=0.1)
    return model

In [15]:
N = 4
metrics_macro = np.zeros((3))
metrics_micro = np.zeros((3))

skf = KFold(n_splits=N, shuffle=True)
for train_index, test_index in skf.split(x, y):
    model_ = train_model(x[train_index],y[train_index])
    preds = model_.predict(x[test_index])
    y_classes = preds.argmax(axis=-1)
    score = model_.evaluate(x[test_index], y[test_index])
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    pred_result = []
    for i in y_classes:
        pred_result.append(relations[i])
    y_list = []
    for i in y[test_index]:
        index = list(i).index(1)
        y_list.append(relations[index])
    
    metrics_macro += precision_recall_fscore_support(y_list, pred_result, average='macro')[:3]
    metrics_micro += precision_recall_fscore_support(y_list, pred_result, average='micro')[:3]

Train on 30240 samples, validate on 3360 samples
Epoch 1/10
30240/30240 [==============================] - 42s 1ms/step - loss: 3.7408 - acc: 0.0851 - val_loss: 8.4884 - val_acc: 0.0241
Epoch 2/10
30240/30240 [==============================] - 34s 1ms/step - loss: 3.3132 - acc: 0.1653 - val_loss: 9.9046 - val_acc: 0.0476
Epoch 3/10
30240/30240 [==============================] - 34s 1ms/step - loss: 3.0852 - acc: 0.2047 - val_loss: 11.4592 - val_acc: 0.0500
Epoch 4/10
30240/30240 [==============================] - 34s 1ms/step - loss: 2.9302 - acc: 0.2374 - val_loss: 12.0692 - val_acc: 0.0500
Epoch 5/10
30240/30240 [==============================] - 35s 1ms/step - loss: 2.8211 - acc: 0.2562 - val_loss: 13.3998 - val_acc: 0.0426
Epoch 6/10
30240/30240 [==============================] - 35s 1ms/step - loss: 2.7329 - acc: 0.2787 - val_loss: 13.4331 - val_acc: 0.0542
Epoch 7/10
30240/30240 [==============================] - 34s 1ms/step - loss: 2.6482 - acc: 0.2966 - val_loss: 13.9971 - val

На деле как-то печальненько...может, конечно, мы недообучились...

In [17]:
metrics_macro/N, metrics_micro/N

(array([0.24967225, 0.27162449, 0.24855736]),
 array([0.27142857, 0.27142857, 0.27142857]))

In [25]:
mixed=[]
for i,y in zip(y_list, pred_result):
    if i!=y:
        mixed.append((i,y))
mixed = Counter(mixed)

In [30]:
mixed.most_common(10)

[(('P551', 'P937'), 39),
 (('P276', 'P931'), 37),
 (('P131', 'P17'), 31),
 (('P1303', 'P106'), 30),
 (('P106', 'P39'), 29),
 (('P800', 'P156'), 29),
 (('P140', 'P1303'), 26),
 (('P57', 'P1877'), 26),
 (('P159', 'P931'), 26),
 (('P27', 'P937'), 25)]

#### Посмотрим на самые частотные ошибки:

'P551', 'P937' 

residence -> work_location

'P276', 'P931' 

location -> place served by transport hub

'P131', 'P17'  

located in the administrative territorial entity -> country

'P1303', 'P106' 

instrument -> occupation 

'P106', 'P39'

occupation ->  position held 

'P800', 'P156' 

notable work  ->  followed by 

'P140', 'P1303'

religion -> instrument

'P57', 'P1877' 

director ->  after a work by 

'P159', 'P931' 

headquarters location  ->  place served by transport hub

'P27', 'P937'  

country of citizenship  -> work_location

Мы видим, что основные ошибки совершаются для типов "location", "occupation"

Если путаницы внутри подклассов "location" и "occupation" более-менее очевидны

Посмотрим на religion -> instrument

In [44]:
for i,m, z in zip(y_list, pred_result, test_index):
    if i=='P140' and m=='P1303':
        print(instances[z])


{'tokens': ['Charles', 'Alexander', ',', 'who', 'became', 'duke', 'in', '1733', ',', 'had', 'become', 'a', 'Roman', 'Catholic', 'while', 'an', 'officer', 'in', 'the', 'Austrian', 'service', '.'], 'h': ['charles alexander', 'Q61946', [[0, 1]]], 't': ['catholic', 'Q1841', [[13]]]}
{'tokens': ['The', 'Anglican', 'Bishop', 'of', 'New', 'Guinea', '(', 'then', 'a', 'diocese', 'of', 'the', 'ecclesiastical', 'Province', 'of', 'Queensland', ')', ',', 'Philip', 'Strong', ',', 'instructed', 'Anglican', 'missionaries', 'to', 'remain', 'at', 'their', 'posts', '.'], 'h': ['philip strong', 'Q7184440', [[18, 19]]], 't': ['anglican', 'Q6423963', [[1], [22]]]}
{'tokens': ['The', 'origins', 'of', 'Evangelicalism', 'are', 'usually', 'traced', 'back', 'to', 'the', 'English', 'Methodist', 'movement', ',', 'Nicolaus', 'Zinzendorf', ',', 'the', 'Moravian', 'Church', ',', 'Lutheran', 'pietism', ',', 'Presbyterianism', 'and', 'Puritanism', '.'], 'h': ['nicolaus zinzendorf', 'Q76336', [[14, 15]]], 't': ['luthera

In [45]:
for i,m, z in zip(y_list, pred_result, test_index):
    if i=='P1303':
        print(instances[z])

{'tokens': ['Robert', 'Drasnin', '(', 'November', '17', ',', '1927', '–', 'May', '13', ',', '2015', ')', 'was', 'an', 'American', 'composer', 'and', 'clarinet', 'player', '.'], 'h': ['robert drasnin', 'Q3435510', [[0, 1]]], 't': ['clarinet', 'Q8343', [[18]]]}
{'tokens': ['The', 'band', 'carried', 'on', 'as', 'a', 'trio', 'with', 'Jon', 'Hiseman', 'on', 'drums', ',', 'but', 'Bond', "'s", 'mental', 'and', 'physical', 'health', 'continued', 'to', 'deteriorate', ',', 'until', 'the', 'band', 'eventually', 'dissolved', 'in', '1967', '.'], 'h': ['jon hiseman', 'Q743051', [[8, 9]]], 't': ['drums', 'Q128309', [[11]]]}
{'tokens': ['Kaderabek', 'remains', 'on', 'the', 'faculty', 'at', 'West', 'Chester', 'University', ',', 'Faculty', 'Profiles', ',', 'Applied', 'Music', ':', 'Frank', 'Kaderabek', 'Instructor', '—', 'Trumpet', '.'], 'h': ['frank kaderabek', 'Q5487621', [[16, 17]]], 't': ['trumpet', 'Q8338', [[20]]]}
{'tokens': ['The', 'trio', 'features', 'Walter', 'Verdehr', 'on', 'violin', ',', 'E

Тут мы видим определенный шаблон: имя собственное -- религия/муз.инструмент, очень похожая структура (хотя я бы сказала, что тут очень не хватает частей речи, которые бы сразу решили эту ситуацию)